### 1. Read Data into DataFrames!

In [1]:
import pandas as pd
from pandasql import *

In [2]:
!ls -l upsell

total 32420
-rw-r----- 1 kevin kevin    37514 Jan  4 19:33 donotcall.csv
-rw-r----- 1 kevin kevin    37514 Jan  4 19:10 donotcall.txt
-rw-rw-r-- 1 kevin kevin    63605 Jan  5 16:56 top1000_transactions_2015.csv
-rw-r----- 1 kevin kevin 14638824 Jan  4 19:33 transactions.csv
-rw-r----- 1 kevin kevin 14638824 Jan  4 19:11 transactions.txt
-rw-r----- 1 kevin kevin  1877331 Jan  4 19:32 users.csv
-rw-r----- 1 kevin kevin  1877331 Jan  4 19:11 users.txt


In [3]:
users_df = pd.read_csv('upsell/users.txt', sep=';', header=True,
                       error_bad_lines=False,
                       names=['Customer_ID', 'name', 'emails', 'phone'])

Skipping line 2079: expected 4 fields, saw 6
Skipping line 5920: expected 4 fields, saw 6
Skipping line 9348: expected 4 fields, saw 6
Skipping line 10764: expected 4 fields, saw 7
Skipping line 15949: expected 4 fields, saw 6
Skipping line 15995: expected 4 fields, saw 7
Skipping line 16549: expected 4 fields, saw 7
Skipping line 18485: expected 4 fields, saw 6
Skipping line 21644: expected 4 fields, saw 7
Skipping line 21845: expected 4 fields, saw 6
Skipping line 23356: expected 4 fields, saw 6



##### Correct the Bad Lines in Data

In [4]:
badlinesnumbers = [2079,5920,9348,10764,15949,
                   15995,16549,18485,21644,21845,23356]

with open('upsell/users.txt') as f:
    badlines_indices = set(map(lambda x : x - 1, badlinesnumbers))
    i = 0
    
    for line in f:
        if i in badlines_indices:
            data = line.strip().split(';')
            data = {'Customer_ID': data[0],
                   'name': data[1],
                   'emails': data[2:-1],
                   'phone': data[-1]}
            users_df.loc[i] = list([data['Customer_ID'], data['name'], ', '.join(data['emails']), data['phone']])
            
        i += 1    
        
users_df.head()

,Customer_ID,name,emails,phone
0,1290614884,Alain Hennesey,"alain.hennesey@facebook.com,alain.hennesey@me.com","(942) 208-8460,(801) 938-2376"
1,1700818057,Hamed Fingerhuth,"hamed.fingerhuth@msn.com,hamed.fingerhuth@me.com",NaN
2,17378782,Annamae Leyte,"annamae.leyte@msn.com,annamae.leyte@facebook.com",NaN
3,1723254379,Chao Peachy,"chao.peachy@me.com,chao.peachy@gmail.com",(510) 121-0098
4,1946358537,Somtochukwu Mouritsen,somtochukwu.mouritsen@me.com,(669) 504-8080


In [5]:
transactions_df = pd.read_csv('upsell/transactions.txt', sep=';',
                              error_bad_lines=False,
                              names=['Customer_ID',
                                     'transaction_amount',
                                     'transaction_date'])
transactions_df.head()

,Customer_ID,transaction_amount,transaction_date
0,815581247,$144.82,2015-09-05
1,1534673027,$140.93,2014-03-11
2,842468364,$104.26,2014-05-06
3,1720001139,$194.60,2015-08-24
4,1397891675,$307.72,2015-09-25


In [6]:
dnc_df = pd.read_csv('upsell/donotcall.txt', names=['number'])
dnc_df.head()

,number
0,(533) 072-2779
1,(942) 208-8460
2,(801) 938-2376
3,(510) 121-0098
4,(669) 504-8080


### Simple Count

In [7]:
print len(users_df), len(transactions_df), len(dnc_df)

24987 500000 2501


### Verify Phone Number Data Format

Q: Do all our numbers in the DNC list have the form (xxx) xxx-xxxx ?

A: they have lengths of 14 characters, but do they have the form (xxx) xxx-xxx ?

In [8]:
phone_number_lengths = dnc_df.number.apply(lambda phone_number : len(phone_number))
(phone_number_lengths - phone_number_lengths[0]).nonzero()

(array([], dtype=int64),)

In [9]:
import re

phone_number_pattern = re.compile("\([0-9]{3}\) [0-9]{3}-[0-9]{4}")

def is_valid_phone_number(phone_number_string):
    return not (phone_number_pattern.match(phone_number_string) is None)

phone_numbers_validity = dnc_df.number.apply(is_valid_phone_number)
phone_numbers_validity.all()

True

Q: Do all our numbers in the DNC list have the form (xxx) xxx-xxxx ?

A: Yes!

### 2. Conduct Analysis!

##### * Objective: We wish to contact users using phone numbers that are not in the do not call list

In [10]:
def phone_list_string_to_list(phone_list_string):
    return str(phone_list_string).split(',')

users_df.phone = users_df.phone.apply(phone_list_string_to_list)
users_df.phone.head()

0    [(942) 208-8460, (801) 938-2376]
1                               [nan]
2                               [nan]
3                    [(510) 121-0098]
4                    [(669) 504-8080]
Name: phone, dtype: object

* remember to format the matching numbers in DNC list

In [11]:
def validate_format_for_phone_number_string_list(phone_number_string_list):
    if not phone_number_string_list or set(['', 'nan']) & set(phone_number_string_list):
        return True # no phone number
    
    return pd.Series([is_valid_phone_number(x) for x in phone_number_string_list]).all()

is_valid_phone_number_format = users_df.phone.apply(validate_format_for_phone_number_string_list)
is_valid_phone_number_format.all()

True

Q: Do all our numbers in the users dataframe have the form (xxx) xxx-xxxx ?

A: Yes!

In [12]:
dnc_set = set([])

for phone_number_list in dnc_df.values:
    for phone_number in phone_number_list:
        dnc_set.add(phone_number)
        
def is_not_in_dnc_list(phone_number):
    return not (phone_number in dnc_set)

In [13]:
def is_prospective_candidate(phone_number_list):
    return pd.Series([is_not_in_dnc_list(phone_number) for phone_number in phone_number_list]).all()

prospective_users_df = users_df[users_df.phone.apply(is_prospective_candidate)]

The selection of the top 1000 users should include the following steps:

1. Selecting the top users based on highest transaction amount for year 2015
2. Save the campaign in a file using the following structure:
Customer ID, Customer name, phone list that can be used to contact the user, total transaction amount

In [32]:
transactions_df['transaction_amount_float'] = transactions_df['transaction_amount'].\
                                              apply(lambda price_str : float(price_str.strip('$')))

In [25]:
pysqldf = lambda q: sqldf(q, globals())

q  = """
SELECT
  t.Customer_ID
  , u.name
  , u.phone
  , t.transaction_amount_float
FROM
  transactions_df t
LEFT JOIN
  prospective_users_df u
ON t.Customer_ID = u.Customer_ID
WHERE
    t.transaction_date > '2014-12-31'
"""

#df = pysqldf(q)
#df.head()

In [34]:
transactions_df.set_index('Customer_ID', inplace=True)
prospective_users_df.set_index('Customer_ID', inplace=True)

In [35]:
candidates_df = transactions_df.join(prospective_users_df, how='inner')
candidates_df = candidates_df.reset_index()
candidates_df.head()

,Customer_ID,transaction_amount,transaction_date,transaction_amount_float,name,emails,phone
0,36547,$192.69,2015-07-16,192.69,Darien Ethel,darien.ethel@msn.com,[nan]
1,36547,$801.49,2015-12-18,801.49,Darien Ethel,darien.ethel@msn.com,[nan]
2,36547,$33.55,2015-11-12,33.55,Darien Ethel,darien.ethel@msn.com,[nan]
3,36547,$101.8,2014-09-08,101.80,Darien Ethel,darien.ethel@msn.com,[nan]
4,36547,$349.3,2014-02-02,349.30,Darien Ethel,darien.ethel@msn.com,[nan]


Selecting users based on transaction amount for year 2015

In [36]:
transactions_2015_df = candidates_df[candidates_df.transaction_date > '2014-12-31']
transactions_2015_df.head()

,Customer_ID,transaction_amount,transaction_date,transaction_amount_float,name,emails,phone
0,36547,$192.69,2015-07-16,192.69,Darien Ethel,darien.ethel@msn.com,[nan]
1,36547,$801.49,2015-12-18,801.49,Darien Ethel,darien.ethel@msn.com,[nan]
2,36547,$33.55,2015-11-12,33.55,Darien Ethel,darien.ethel@msn.com,[nan]
6,36547,$65.67,2015-06-19,65.67,Darien Ethel,darien.ethel@msn.com,[nan]
8,36547,$476.14,2015-07-23,476.14,Darien Ethel,darien.ethel@msn.com,[nan]


In [37]:
def calculate_total_transaction_amount(sub_dataframe_by_customer_id):
    sub_dataframe_by_customer_id['total_transaction_amount'] = sub_dataframe_by_customer_id['transaction_amount'].\
                                                               apply(lambda price_str : float(price_str.strip('$'))).\
                                                               sum()
    return sub_dataframe_by_customer_id

total_transaction_amount_by_customer = transactions_2015_df[['Customer_ID', 'transaction_amount']].\
                                       groupby('Customer_ID').\
                                       apply(calculate_total_transaction_amount)

In [38]:
total_transaction_amount_by_customer = total_transaction_amount_by_customer[['Customer_ID', 'total_transaction_amount']].\
                                       groupby('Customer_ID').\
                                       mean()
        
total_transaction_given_customer_id = dict(zip(total_transaction_amount_by_customer.index,
                                               total_transaction_amount_by_customer.total_transaction_amount.values))

In [39]:
transactions_2015_df['total_transaction_amount'] = transactions_2015_df['Customer_ID'].\
                                                   apply(lambda cid : total_transaction_given_customer_id[cid])

/home/kevin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [42]:
transactions_2015_df['phone'] = transactions_2015_df['phone'].apply(tuple)

/home/kevin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [137]:
result_df = transactions_2015_df[['Customer_ID', 'name', 'phone', 'total_transaction_amount']].drop_duplicates()

In [138]:
result_df.sort(['total_transaction_amount'],
               ascending=False,
               inplace=True)

In [139]:
TOP_N = 1000

In [140]:
result_df['phone'] = result_df['phone'].apply(lambda phone_number_list : str(list(phone_number_list)).strip('[]').replace(', ', ',').replace("'",""))
result_df['total_transaction_amount'] = result_df['total_transaction_amount'].apply(lambda x : '$' + str(x))

import numpy as np
result_df = result_df.drop(result_df[result_df.phone.apply(lambda phone_list : 'nan' in phone_list)].index)

In [142]:
result_df[:TOP_N].to_csv('upsell/top1000_transactions_2015.csv',
                         header=False,
                         index=False,
                         sep=';')

In [143]:
!head -20 upsell/top1000_transactions_2015.csv

1295065819;Bryanne Stopp;(217) 541-9868,(531) 246-6723;$15618.61
987109832;Kaidence Guernsey;(592) 799-6134;$12863.54
460093729;Hunter Craggs;(244) 021-2312;$12671.06
563381140;Vardaan Prestwich;(012) 207-3502;$12560.74
1471106934;Seeley Boak;(149) 211-1707;$12025.77
1153503828;Vasily Georgii;(366) 412-3329;$11812.18
1959653011;Angelisa Cummings;(185) 668-0124;$11559.2
693794122;Credence Sherard;(453) 636-8177;$11410.21
944173610;Denilson Peattie;(623) 477-3650;$11366.88
707056747;Nasteha Bister;(016) 015-6096,(877) 013-4794;$11358.05
2049030516;Kelbie Puppe;(089) 988-0109,(569) 457-0291;$11315.68
317427146;Stevie Hooley;(181) 449-9765,(634) 039-1950;$11311.92
104050028;Weatherly Shannon;(869) 359-2405,(517) 263-5875;$11311.41
681405035;Brysten Jeffs;(184) 202-3346;$11310.42
697077690;Rakyla Goode;(889) 863-5817,(308) 012-4466;$11296.85
1438177405;Analaura Beetham;(668) 324-0222;$11244.02
2014804806;Davida Notman;(956) 421-9300;$11217.17
1954941481;Karsten Collinge;(466) 244-5295;$1114